# Natural Language Processing project "Diacritic Restoration"
## Muhammet Serdar NAZLI, Hasan Taha BAĞCI

### Make sure you have at least 4GB of VRAM. 

## Data Preprocessing

### Do you want to use external datasets for training?

In [1]:
use_external_data_wiki = False          # https://www.kaggle.com/datasets/omercolakoglu/10m-rows-fake-turkish-names-and-address-dataset# https://huggingface.co/datasets/emrecan/stsb-mt-turkish/raw/main/train.csv, https://www.kaggle.com/datasets/mahdinamidamirchi/turkish-sentences-dataset?select=wiki.tr.txt
use_external_data_news = False          # https://www.kaggle.com/datasets/furkanozbay/turkish-news-dataset?select=news.xls

In [2]:
from data_loader.constantsForData import mapping, to_removed_chars, conversion_dict, reverse_conversion_dict
from data_loader.utils import expand_sentences, split_long_sentences, tr_upper, tr_lower, split_exact_row, preprocess_text

import pandas as pd
import torch 
import numpy as np 
from torch.utils.data import Dataset, DataLoader 



train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
if use_external_data_wiki:
    new_data = pd.read_csv('new_data.csv') 
    new_data['concat'] = new_data['sentence1'] + new_data['sentence2']      
    concat = np.concatenate([new_data['sentence1'].to_numpy(), new_data['sentence2'].to_numpy()])
    concat = np.concatenate([train_df['Sentence'].to_numpy(),concat])
    train_df = pd.DataFrame(concat, columns=['Sentence']) 
    train_df["ID"] = train_df.index
    train_df.dropna(inplace=True)

if use_external_data_news:
    news_data = pd.read_csv('news.csv')
    news = news_data['content'].to_numpy() 
    concat = np.concatenate([train_df['Sentence'].to_numpy(),news])
    train_df = pd.DataFrame(concat, columns=['Sentence'])
    train_df["ID"] = train_df.index
    train_df.dropna(inplace=True)


row_index = 15480                               # 15480, this sentence is too long, split it into parts where each part is leq tokens      
train_df = split_exact_row(row_index, train_df)

# Expand both train and test DataFrames
train_df = expand_sentences(train_df)

# Strip test_df Sentence column of leading/trailing whitespaces 
test_df['Sentence'] = test_df['Sentence'].str.strip()

In [3]:
train_df.ID = train_df.index
train_df

,ID,Sentence
0,0,"sınıf , havuz ve açık deniz çalışmalarıyla , t..."
1,1,"bu standart , sualtında kendini rahat hisseden..."
2,2,"yapılan araştırmalar , öğrencilerin mevcut dal..."
3,3,"pdıc öğrencilerinde , psikolojik eğitim ve yet..."
4,4,"pdıc eğitiminin sağladığı güven ve rahatlık , ..."
...,...,...
52756,52756,"banka , müşteri adına müşterinin talep ettiği ..."
52757,52757,ancak müşterinin bankaya borcunun karşılığı ol...
52758,52758,müşteri bankaya borcunu belli bir ödeme planın...
52759,52759,uzun vadeli olarak insanları konut sahip yapma...


In [4]:
train_df["Poisoned_tokens"] = train_df["Sentence"].apply(lambda x: preprocess_text(x, conversion_dict, to_removed_chars, mapping, poison=True))
train_df["Sentence_tokens"] = train_df["Sentence"].apply(lambda x: preprocess_text(x, conversion_dict, to_removed_chars, mapping))

test_df["Sentence_tokens"] = test_df["Sentence"].apply(lambda x: preprocess_text(x, conversion_dict, to_removed_chars, mapping, is_test=True))

In [5]:
display(train_df.head())
display(test_df.head())

,ID,Sentence,Poisoned_tokens,Sentence_tokens
0,0,"sınıf , havuz ve açık deniz çalışmalarıyla , t...","[52, 42, 47, 42, 39, 0, 11, 0, 41, 34, 55, 54,...","[52, 69, 47, 69, 39, 0, 11, 0, 41, 34, 55, 54,..."
1,1,"bu standart , sualtında kendini rahat hisseden...","[35, 54, 0, 52, 53, 34, 47, 37, 34, 51, 53, 0,...","[35, 54, 0, 52, 53, 34, 47, 37, 34, 51, 53, 0,..."
2,2,"yapılan araştırmalar , öğrencilerin mevcut dal...","[58, 34, 49, 42, 45, 34, 47, 0, 34, 51, 34, 52...","[58, 34, 49, 69, 45, 34, 47, 0, 34, 51, 34, 70..."
3,3,"pdıc öğrencilerinde , psikolojik eğitim ve yet...","[49, 37, 42, 36, 0, 48, 40, 51, 38, 47, 36, 42...","[49, 37, 69, 36, 0, 66, 68, 51, 38, 47, 36, 42..."
4,4,"pdıc eğitiminin sağladığı güven ve rahatlık , ...","[49, 37, 42, 36, 0, 38, 40, 42, 53, 42, 46, 42...","[49, 37, 69, 36, 0, 38, 68, 42, 53, 42, 46, 42..."


,ID,Sentence,Sentence_tokens
0,0,tr ekonomi ve politika haberleri turkiye nin e...,"[53, 51, 0, 38, 44, 48, 47, 48, 46, 42, 0, 55,..."
1,1,uye girisi,"[54, 58, 38, 0, 40, 42, 51, 42, 52, 42]"
2,2,son guncelleme 12:12,"[52, 48, 47, 0, 40, 54, 47, 36, 38, 45, 45, 38..."
3,3,Imrali Mit gorusmesi ihtiyac duyuldukca oluyor,"[42, 46, 51, 34, 45, 42, 0, 46, 42, 53, 0, 40,..."
4,4,Suriye deki silahli selefi muhalifler yeni kur...,"[52, 54, 51, 42, 58, 38, 0, 37, 38, 44, 42, 0,..."


In [6]:
test_df

,ID,Sentence,Sentence_tokens
0,0,tr ekonomi ve politika haberleri turkiye nin e...,"[53, 51, 0, 38, 44, 48, 47, 48, 46, 42, 0, 55,..."
1,1,uye girisi,"[54, 58, 38, 0, 40, 42, 51, 42, 52, 42]"
2,2,son guncelleme 12:12,"[52, 48, 47, 0, 40, 54, 47, 36, 38, 45, 45, 38..."
3,3,Imrali Mit gorusmesi ihtiyac duyuldukca oluyor,"[42, 46, 51, 34, 45, 42, 0, 46, 42, 53, 0, 40,..."
4,4,Suriye deki silahli selefi muhalifler yeni kur...,"[52, 54, 51, 42, 58, 38, 0, 37, 38, 44, 42, 0,..."
...,...,...,...
1152,1152,Yuregir Adana ilimize ait sirin bir ilcedir,"[58, 54, 51, 38, 40, 42, 51, 0, 34, 37, 34, 47..."
1153,1153,yuze guluculugun at oynattigi bir aydinlar ort...,"[58, 54, 59, 38, 0, 40, 54, 45, 54, 36, 54, 45..."
1154,1154,zavalli adami oracikta astilar ve hic kimse se...,"[59, 34, 55, 34, 45, 45, 42, 0, 34, 37, 34, 46..."
1155,1155,zengin cocuklarina ariz munasebetsizlikler fak...,"[59, 38, 47, 40, 42, 47, 0, 36, 48, 36, 54, 44..."


In [7]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    if type(batch[0]) is tuple:
        original_tokens, poisoned_tokens = zip(*batch)
        original_tokens_padded = pad_sequence(original_tokens, batch_first=True, padding_value=conversion_dict['[PAD]'])
        poisoned_tokens_padded = pad_sequence(poisoned_tokens, batch_first=True, padding_value=conversion_dict['[PAD]'])
        return original_tokens_padded, poisoned_tokens_padded
    else:
        original_tokens = pad_sequence(batch, batch_first=True, padding_value=conversion_dict['[PAD]'])
        return original_tokens


### Not enough VRAM? decrease batch size. 256 batch size -> 11GB VRAM, 64 batch size -> 4GB VRAM

In [8]:
from torch.utils.data import Dataset, DataLoader
from data_loader.data_loaders import SentenceDataset, SentenceDatasetTest

# Create the dataset and dataloader
train_dataset = SentenceDataset(train_df)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn)
    
# Create the dataset for the test data
test_dataset = SentenceDatasetTest(test_df['Sentence_tokens'].tolist())
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)


## Model

In [9]:
from model.trainer import train_model, predict
from model.model import LSTMModel
from model.loss import FocalLoss 

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(len(conversion_dict))

105


In [11]:
model = LSTMModel(vocab_size=len(conversion_dict), embedding_dim=64, hidden_dim=128, output_dim=len(conversion_dict), num_layers=4, dropout_rate=0.3)
model.to(device)
criterion = torch.nn.CrossEntropyLoss(label_smoothing=1e-3, ignore_index=conversion_dict['[PAD]'])
#criterion = FocalLoss(ignore_index=conversion_dict['[PAD]'])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
# model.load_state_dict(torch.load('model_best.pth')) 

/home/nserd/anaconda3/envs/nlp_project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Below, you can skip the training procedure and directly load the trained model and use it for inference.

### Model best chkpt link: https://drive.google.com/file/d/1ujwNiR5sd5KryogZ9qYCnGVoY_z6El7B/view?usp=sharing

In [22]:
#train_model(train_dataloader, model, criterion, optimizer, epochs=10, device=device, save_best_model=True)

Epoch 1:  31%|███▏      | 65/208 [00:20<00:45,  3.11it/s, loss=3.12]

In [12]:
use_pretrained_model = True 
if use_pretrained_model:
    model.load_state_dict(torch.load('model_best.pth')) 

In [13]:
predictions = predict(model, test_dataloader, reverse_conversion_dict, device=device, sos_eos_tokens=False)

Predicting: 100%|██████████| 1157/1157 [00:03<00:00, 342.95it/s]


In [14]:
predictions

['tr ekonomi ve politika haberleri türkiye nin en cesur gazetesi radikal de üye ol',
 'üye girişi',
 'son güncelleme 12:12',
 'imralı mit görüşmesi ihtiyaç duyuldukça oluyor',
 'suriye deki silahlı selefi muhalifler yeni kurulan suriye muhalif ve devrimci güçler ulusal koalisyonunu tanımadıklarını duyurdu',
 'ancak ölüm haberleri savaştan çok tek taraflı bir kayıp verme süreci yaşandığını gösteriyor',
 'israil in 4 üncü gününe giren gazze saldırılarında bir f 16 savaş uçağının hamas tarafından düşürüldüğü ve pilotlarının da esir alındığı ileri sürüldü',
 'serbes: memecan sen mizahçı mısın',
 'müslüm gürses yoğun bakımda',
 'takip et: wwwradikalcomtr',
 '34 yaşındaki hakan polat göğüs ve kol ağırısı şikayetiyle gittiği hastanede tedavinin ardından taburcu olmak için işlemlerini yaptırırken yaşamını yitirdi',
 'sarı lacivertli kulüp yandex ile fenerbahçe temalı internet tarayıcısı anlaşması imzaladı',
 'kanada nın fransız kültürünün baskın olduğu quebec eyaleti önümüzdeki günlerde dünyan

In [15]:
# https://stackoverflow.com/questions/19703106/python-and-turkish-capitalization
# Use unicode_
def fix_casing(original, predicted):
    fixed = []
    for orig, pred in zip(original, predicted):
        if orig.isupper():
            fixed.append(tr_upper(pred))
        else:
            fixed.append(pred)
    return ''.join(fixed)

fixed_predictions = [fix_casing(test_df.iloc[i]['Sentence'], predictions[i]) for i in range(len(predictions))]

In [16]:
fixed_predictions

['tr ekonomi ve politika haberleri türkiye nin en cesur gazetesi radikal de üye ol',
 'üye girişi',
 'son güncelleme 12:12',
 'İmralı Mit görüşmesi ihtiyaç duyuldukça oluyor',
 'Suriye deki silahlı selefi muhalifler yeni kurulan Suriye muhalif ve devrimci güçler ulusal koalisyonunu tanımadıklarını duyurdu',
 'ancak ölüm haberleri savaştan çok tek taraflı bir kayıp verme süreci yaşandığını gösteriyor',
 'İsrail in 4 üncü gününe giren Gazze saldırılarında bir f 16 savaş uçağının Hamas tarafından düşürüldüğü ve pilotlarının da esir alındığı ileri sürüldü',
 'Serbes: Memecan sen mizahçı mısın',
 'Müslüm Gürses yoğun bakımda',
 'takip et: wwwradikalcomtr',
 '34 yaşındaki Hakan Polat göğüs ve kol ağırısı şikayetiyle gittiği hastanede tedavinin ardından taburcu olmak için işlemlerini yaptırırken yaşamını yitirdi',
 'sarı lacivertli kulüp Yandex ile Fenerbahçe temalı internet tarayıcısı anlaşması imzaladı',
 'Kanada nın Fransız kültürünün baskın olduğu quebec eyaleti önümüzdeki günlerde dünyan

In [18]:
submission = pd.DataFrame({'ID': test_df['ID'], 'Sentence': fixed_predictions})

In [19]:
submission 

,ID,Sentence
0,0,tr ekonomi ve politika haberleri türkiye nin e...
1,1,üye girişi
2,2,son güncelleme 12:12
3,3,İmrali Mit görüşmesi ihtiyaç duyuldukça oluyor
4,4,Suriye deki silahli selefi muhalifler yeni kur...
...,...,...
1152,1152,Yüreğir Adana ilimize ait sırın bir ilçedir
1153,1153,yüze gülücülüğün at oynattığı bir aydınlar ort...
1154,1154,zavallı adamı oracıkta aştılar ve hiç kimse se...
1155,1155,zengin çocuklarına arız munasebetsizlikler fak...


In [20]:
submission.to_csv('submission.csv', index=False) 

In [19]:
device = 'cpu'

In [20]:
model.to(device)
def try_a_sentence(sentence:str):
    orig_sentence = sentence
    sentence = preprocess_text(sentence, conversion_dict, to_removed_chars, mapping)
    sentence = torch.tensor(sentence).unsqueeze(0)  # Add batch dimension
    sentence.to(device) 
    model.eval()
    with torch.no_grad():
        output = model(sentence)
    output = torch.argmax(output, dim=-1).squeeze(0)
    output = output.tolist()
    output = [reverse_conversion_dict[token] for token in output]
    # convert output to numpy array 
    output = np.array(output)
    return fix_casing(orig_sentence, output)


In [21]:
sentence = "ITU NLP grubu, Turkce dogal dıl ısleme alaninda calisiyor"
try_a_sentence(sentence)

'İTÜ NLP grubu, Türkçe doğal dil işleme alanında çalışıyor'

In [22]:
sentence = "cekoslavakyalilastiramadiklarimizdanmissinizcasina"
try_a_sentence(sentence)

'çekoslavakyalılaştıramadıklarımızdanmışsınızcasına'

In [23]:
sentence = "Rusyadan gelen arkadasim donmus ve orada  balik yememis"
try_a_sentence(sentence)

'Rusyadan gelen arkadaşım donmuş ve orada  balık yememiş'

In [24]:
sentence = "ITU NLP grubu uyelerini, DRL donem odevınde yenecegiz."
try_a_sentence(sentence)

'İTÜ NLP grubu üyelerini, DRL dönem ödevinde yeneceğiz.'

In [25]:
sentence = "Muvaffakiyetsizlestiricilestiri veremeyebileceklerimizdenmissinizcesine"
try_a_sentence(sentence)

'Muvaffakiyetsizleştiricileştiri veremeyebileceklerimizdenmişsinizcesine'

In [26]:
s = "Donarak olmek ıstemıyorum"
try_a_sentence(s)

'Donarak ölmek istemiyorum'

In [27]:
s = "donerek dusmek istemiyorum"
try_a_sentence(s)

'dönerek düşmek istemiyorum'

In [28]:
s = "unlu yazar, donerek camdan dustu; babam, olmusla olmuse care yok dedi."
try_a_sentence(s)

'ünlü yazar, dönerek camdan düştü; babam, olmuşla ölmüşe çare yok dedi.'

In [29]:
s = "unlu yazar, donerek camdan dustu; babam, olmus la olmuse care yok dedi."
try_a_sentence(s)

'ünlü yazar, dönerek camdan düştü; babam, olmuş la ölmüşe çare yok dedi.'

In [30]:
s = "donarak veya donerek olmek istemiyorum, donebilmek icin donmaliyim"
try_a_sentence(s)

'donarak veya dönerek ölmek istemiyorum, dönebilmek için donmalıyım'

In [31]:
s = "donarak dusmek istemiyorum, dusunebilmek için donmalıyım ama dusersem donabilirim."

try_a_sentence(s)

'donarak düşmek istemiyorum, düşünebilmek için donmalıyım ama düşersem donabılirim.'

In [32]:
sentence = "NLP modeli neden terapiye gitmis? Cunku cok fazla cozulmemis cumlesi varmıs!"
result = try_a_sentence(sentence) 
print(result)  

NLP modeli neden terapiye gitmiş? Çünkü çok fazla çözülmemiş cümlesi varmış!


In [36]:
sentence = "NLP modeli neden bulmacayı cozmeyi birakti? cunku her cumlenin sonunu tahmin etmeye calisirken basi donmustu!"
try_a_sentence(sentence)

'NLP modeli neden bulmacayı çözmeyi bıraktı? çünkü her cümlenin sonunu tahmin etmeye çalışırken başı dönmüştür'

In [49]:
s = "kirsehir'e gitmek icin once bozuyuk sonra kutahyaya gitmek gerekiyor"
try_a_sentence(s)

"kırşehir'e gitmek için önce bozuyuk sonra kütahyaya gitmek gerekiyor"